# Translated Periodicity using a simple Block
The mesh shown below has periodic faces on the z and y axis. The code below shows how we find the translated periodicity in the y and z directions.

Steps Overview
1. Find Connectivity - This splits the faces of all the blocks so that they match 
2. Build the Block connectivity matrix using connectivity - Uses the split faces to determine which blocks are connected. value of 1 = connected, 0 = not searched yet, -1 = not connected 
3. Search for connected faces on the z and y axis - Uses the block connectivity to find the outer connected faces. So, all the top faces for instance. 
4. Find translated periodicity - Uses the connected faces e.g. the left and right, or top and bottom, or front and back. Checks the connected faces to see if any of them match faces from the other side. 

![image.png](tutorial_images/block-overview.png)

In [9]:
# Install Plot3D
!pip install plot3d

In [1]:
# Import functions
import os, pickle
from plot3d import read_plot3D, connectivity_fast,translational_periodicity, write_plot3D, Direction, split_blocks, block_connection_matrix, find_bounding_faces
from plot3d import outer_face_dict_to_list, match_faces_dict_to_list
import numpy as np
data_file = 'block_data.pickle'
mesh_file = 'iso65_64blocks.xyz'
#%% Find connectivity 
def dump_data(data):
    with open('block_data.pickle','wb') as f:
        pickle.dump(data,f)

def read_data():
    with open('block_data.pickle','rb') as f:
        return pickle.load(f)


ModuleNotFoundError: No module named 'plot3d'

In [12]:
# Load the mesh
blocks = read_plot3D(mesh_file,True)
n = len(blocks)
print(f'Number of blocks: {n}')

Number of blocks: 64


## Step 1: Find Connectivity
The code below finds the connectivity and creates the pickle file storing the connectivity 

In [13]:
print('Finding connectivity')
face_matches, outer_faces = connectivity_fast(blocks)
[m.pop('match',None) for m in face_matches] # Remove the dataframe
print('Organizing split and outerfaces')
all_faces = match_faces_dict_to_list(blocks,face_matches)
all_faces.extend(outer_face_dict_to_list(blocks,outer_faces))
all_faces = [m.to_dict() for m in all_faces]
data = {
            "face_matches":face_matches, 
            "outer_faces":outer_faces,
            "all_faces":all_faces
        }
dump_data(data)

Finding connectivity
gcd to use 16


Checking connections block 62 with 63: 100%|██████████| 363/363 [00:14<00:00, 24.22it/s]


Organizing split and outerfaces


## Step 2: Build Block to Block connectivity matrix
Block to block connectivity makes it easier to search for which blocks are connected. For example `print(c[10,4])` if this is 1 then that means blocks 10 and 4 are touching. 

In [14]:
print('Creating block connection matrix')
c = block_connection_matrix(blocks,all_faces)
data["connectivity_matrix"]=c
dump_data(data)

Creating block connection matrix


Building block to block connectivity matrix: checking 62: 100%|██████████| 2016/2016 [00:02<00:00, 881.12it/s]


These are the blocks where block 2 is connected to. We have blocks 0,3,6, and 16. Block 2 is automtically connected with itself. 

In [16]:
print("connectivity matrix shape: " + str(c.shape[0]) + ' by ' + str(c.shape[1]))
np.where(c[2,:]==1)

connectivity matrix shape: 64 by 64


(array([ 0,  2,  3,  6, 16], dtype=int64),)

## Step 3: Find bounding faces
These are faces on either the left or right side of the block.

In [20]:
data = read_data()    
all_faces = data['all_faces']
connectivity_matrix = data['connectivity_matrix']

#%% Find bounding Faces
forward_bound, backward_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"x")
lower_bound, upper_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"z")
left_bound, right_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"y")
data['forward_bound'] = forward_bound
data['backward_bound'] = backward_bound
data['lower_bound'] = lower_bound
data['upper_bound'] = upper_bound
data['left_bound'] = left_bound
data['right_bound'] = right_bound
dump_data(data)

## Step 4: Find Periodicity
In this step we determine the connected faces in the y-direction as well as the z-direction. The program will automatically inform the user if it's not periodic due to not all faces matching. If this is the case then the faces should be plotted using paraview and the user needs to check with their grid generator or manually add in the connectivity. 

In [21]:
#%% Use bounding faces to find periodicity
data = read_data()
forward_bound = data['forward_bound']; backward_bound = data['backward_bound']
lower_bound = data['lower_bound']; upper_bound = data['upper_bound']
left_bound = data['left_bound']; right_bound = data['right_bound']
x_periodic_faces_export, periodic_faces = translational_periodicity(blocks,forward_bound,backward_bound,translational_direction='x')
y_periodic_faces_export, periodic_faces = translational_periodicity(blocks,left_bound,right_bound,translational_direction='y')
z_periodic_faces_export, periodic_faces = translational_periodicity(blocks,lower_bound,upper_bound,translational_direction='z')
data['x_periodic'] = x_periodic_faces_export
data['z_periodic'] = z_periodic_faces_export
data['y_periodic'] = y_periodic_faces_export
dump_data(data)

Checking connections block 32 with 41: 100%|██████████| 16/16 [00:00<00:00, 29.74it/s]



Periodic x


Checking connections block 27 with 63: 100%|██████████| 16/16 [00:00<00:00, 34.63it/s]


check

Periodic y


Checking connections block 45 with 63: 100%|██████████| 16/16 [00:00<00:00, 22.69it/s]


Periodic z


# Export Data to GlennHT connectivity format

In [22]:
blocks = read_plot3D(mesh_file,True)

In [27]:
from glennht_con import export_to_glennht_conn
data = read_data()
matches = data['x_periodic']
matches.extend(data['y_periodic'])
matches.extend(data['z_periodic'])

outer_faces = data['outer_faces']

match_faces = match_faces_dict_to_list(blocks,matches)
match_faces.extend(match_faces_dict_to_list(blocks,data['face_matches']))

outer_faces = outer_face_dict_to_list(blocks,outer_faces)

outer_faces = [o.to_dict() for o in outer_faces if o not in match_faces]
data['outer_faces_final'] = outer_faces
dump_data(data)
export_to_glennht_conn(matches=matches,block_surfaces=outer_faces,filename='block')